In [2]:
#查看一下原文件下的图片大小均为几何
from PIL import Image
import os

# 指定包含图片的文件夹路径
folder_path = r""E:\Code\GAN\generated_images"
# folder_path = r"E:\Code\GAN\data\Plaster_side"

# 获取文件夹中所有的文件
file_list = os.listdir(folder_path)

# 遍历文件夹中的每个文件
for file_name in file_list:
    # 确保文件是tif格式的图片
    if file_name.endswith(".tif"):
        # 构建完整的文件路径
        file_path = os.path.join(folder_path, file_name)

        # 打开图片
        img = Image.open(file_path)

        # 获取图片的大小
        width, height = img.size
        if width != 548:
            print('图片宽度存在例外{}'.format(width))
        if height != 822:
            print('图片高度存在例外{}'.format(height)) 

        # 关闭图片
        img.close()
                       
# 打印图片的大小
print(f"普遍宽度: {width} 像素, 普遍高度: {height} 像素")


图片宽度存在例外500
图片高度存在例外334
普遍宽度: 500 像素, 普遍高度: 334 像素


In [15]:
import torch
import tempfile
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
from pytorch_fid import fid_score
import cv2
import optuna
from torch.utils.tensorboard import SummaryWriter

In [16]:
# 设置GPU设备（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
# 数据预处理
transform = transforms.Compose([
#     transforms.Resize((224, 224)),  # 调整图片大小
    transforms.ToTensor(),         # 转换为张量
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 归一化
])

In [18]:
# 数据加载和预处理

class CustomDataset(Dataset):
    def __init__(self, data_root, transform=None):
        self.data_root = data_root
        self.image_files = [f for f in os.listdir(data_root) if f.lower().endswith(".tif")]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.data_root, self.image_files[idx])

        try:
            image = Image.open(img_name).convert("RGB")

            if self.transform:
                image = self.transform(image)

            # 返回数据（不返回标签）
            return image
        except Exception as e:
            print(f"Error loading image {img_name}: {e}")
            # 返回一个空tensor，表示数据加载失败
            return torch.empty((3, 64, 64))

def get_data_loader(data_root, batch_size):
    transform = transforms.Compose([
#         transforms.Resize(224),
#         transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    dataset = CustomDataset(data_root, transform=transform)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    return data_loader

In [20]:
# 加载自定义数据集
custom_dataset = CustomDataset(data_root="E:/Code/GAN/data/Plaster_side", transform=transform)
# DataLoader用于批量处理
dataloader = DataLoader(custom_dataset, batch_size=64, shuffle=True, num_workers=0)

In [21]:
# 定义生成器
# 可以根据需要调整ngf、nz、nc、ndf和其他超参数来满足你的需求。这些参数控制着模型的深度和宽度，可以影响生成图片的质量。
class Generator(nn.Module):
    def __init__(self, ngf=64, nz=100, nc=3):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)


In [22]:
# 定义判别器
# 可以根据需要调整ngf、nz、nc、ndf和其他超参数来满足你的需求。这些参数控制着模型的深度和宽度，可以影响生成图片的质量。
class Discriminator(nn.Module):
    def __init__(self, ndf=64, nc=3):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [23]:
# 创建SummaryWriter来记录损失到TensorBoard
writer = SummaryWriter(log_dir=f"runs")

In [24]:
# 在这里你可以计算你的优化指标，如生成图片的质量等
real_images_folder_path = "E:/Code/GAN/data/Plaster_side"

def compute_fid(generator, num_samples=100, device='cuda'):
    # 创建临时目录用于保存生成的图片
    temp_dir = tempfile.mkdtemp()

    # 生成num_samples张图片并保存到临时目录
    with torch.no_grad():
        for i in range(num_samples):
            noise = torch.randn(1, 100, 1, 1).to(device)
            fake_image = generator(noise)
            fake_image = fake_image[0].cpu().numpy()
            fake_image = (fake_image + 1.0) / 2.0  # 将像素值范围从[-1, 1]转换到[0, 1]
            fake_image = (fake_image * 255).astype(np.uint8)  # 转换为整数类型
            fake_image = np.transpose(fake_image, (1, 2, 0))  # 将通道维度移到最后
            image_filename = os.path.join(temp_dir, f'fake_image_{i}.png')
            cv2.imwrite(image_filename, fake_image)

    # 计算FID指标
    fid = fid_score.calculate_fid_given_paths(
        [str(real_images_folder_path), temp_dir],  # 真实图片文件夹和生成图片文件夹的路径
        batch_size=50,
        device=device,
        dims=2048  # 这个值应该与 Inception V3 模型的输出维度匹配
    )

    # 删除临时生成的图片
    for i in range(num_samples):
        image_filename = os.path.join(temp_dir, f'fake_image_{i}.png')
        os.remove(image_filename)
    os.rmdir(temp_dir)  # 删除临时目录

    return fid


In [25]:
# 定义超参数搜索空间
def objective(trial):
    # 定义超参数搜索范围
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    ngf = trial.suggest_int('ngf', 32, 256)
    ndf = trial.suggest_int('ndf', 32, 256)

    # 创建生成器和判别器
    generator = Generator(ngf=ngf).to(device)
    discriminator = Discriminator(ndf=ndf).to(device)

    # 定义优化器
    optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    # 定义损失函数
    criterion = nn.BCELoss()
    
    # 学习率调度器，这里使用 StepLR 调度器作为示例
    scheduler_G = torch.optim.lr_scheduler.StepLR(optimizer_G, step_size=30, gamma=0.1)
    scheduler_D = torch.optim.lr_scheduler.StepLR(optimizer_D, step_size=30, gamma=0.1)

    # 训练模型
    num_epochs = 100
    for epoch in range(num_epochs):
        for batch_idx, real_images in enumerate(dataloader):
            real_images = real_images.to(device)
            batch_size = real_images.size(0)
            fake_labels = torch.zeros(batch_size, 1).to(device)

            # 训练判别器
            optimizer_D.zero_grad()
            real_outputs = discriminator(real_images)

            # 创建与 real_outputs 相同维度的 real_labels
            real_labels = torch.ones_like(real_outputs).to(device)

            real_loss = criterion(real_outputs, real_labels)
            real_loss.backward()

            noise = torch.randn(batch_size, 100, 1, 1).to(device)
            fake_images = generator(noise)
            fake_outputs = discriminator(fake_images.detach())

            # 创建与 fake_outputs 相同维度的 fake_labels
            fake_labels = torch.zeros_like(fake_outputs).to(device)

            fake_loss = criterion(fake_outputs, fake_labels)
            fake_loss.backward()
            optimizer_D.step()

            # 训练生成器
            optimizer_G.zero_grad()
            fake_outputs = discriminator(fake_images)

            # 创建与 fake_outputs 相同维度的 real_labels（因为生成器希望生成的图像被判别为真实的）
            real_labels = torch.ones_like(fake_outputs).to(device)

            g_loss = criterion(fake_outputs, real_labels)
            g_loss.backward()
            optimizer_G.step()
            
#              # 记录损失到TensorBoard
#             global_step = epoch * len(dataloader) + batch_idx
#             writer.add_scalar('Loss/Real Loss', real_loss.item(), global_step)
#             writer.add_scalar('Loss/Fake Loss', fake_loss.item(), global_step)
#             writer.add_scalar('Loss/Generator Loss', g_loss.item(), global_step)
    
        # 学习率调度器更新学习率
        scheduler_G.step()
        scheduler_D.step()
        
    # 计算你的优化指标，如生成图片的质量等
    fid = compute_fid(generator, num_samples=100, device=device)

    return fid  # 返回FID作为优化指标


In [26]:
# 创建Optuna study对象
study = optuna.create_study(direction='minimize')  # 我们将FID视为需要最小化的指标

[I 2023-08-30 09:09:45,076] A new study created in memory with name: no-name-c69f4712-94a7-49d3-8e40-5af568e6fd9b


In [27]:
# 开始超参数优化
study.optimize(objective, n_trials=10)  # 你可以调整n_trials来控制搜索次数

C:\Users\admin\AppData\Local\Temp\ipykernel_9148\4116395331.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.43it/s]
[I 2023-08-30 09:10:28,499] Trial 0 finished with value: 435.47806486800806 and parameters: {'lr': 0.0009269545783490616, 'batch_size': 64, 'ngf': 187, 'ndf': 141}. Best is trial 0 with value: 435.47806486800806.
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.43it/s]
[I 2023-08-30 09:11:15,292] Trial 1 finished with value: 410.89280127008675 and parameters: {'lr': 0.00860859467689019, 'batch_size': 64, 'ngf': 69, 'ndf': 244}. Best is trial 1 with value: 410.89280127008675.
100%|███████████

In [28]:
# 输出最佳超参数和最佳指标值,optuna只是寻找出了最佳参数组合，而并没有对模型进行训练
print("Best trial:")
best_trial = study.best_trial
print(f"Best FID: {best_trial.value}")
print("Best Params: ")
for key, value in best_trial.params.items():
    print(f"{key}: {value}")

Best trial:
Best FID: 382.85316126593574
Best Params: 
lr: 0.0013117504668070051
batch_size: 64
ngf: 56
ndf: 244


In [29]:
# 创建生成器和判别器模型（使用最佳超参数）
best_ngf = best_trial.params['ngf']
best_ndf = best_trial.params['ndf']
best_lr = best_trial.params['lr']
# best_lr = 0.5
#使用手动调整，optuna计算出来的学习率可能偏小

# 创建生成器和判别器模型（使用最佳超参数）
generator = Generator(ngf=best_ngf).to(device)
discriminator = Discriminator(ndf=best_ndf).to(device)

In [30]:
# 定义优化器和损失函数
optimizer_G = optim.Adam(generator.parameters(), lr=best_lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=best_lr, betas=(0.5, 0.999))
criterion = nn.BCELoss()

In [31]:
# 学习率调度器，这里使用 StepLR 调度器作为示例
scheduler_G = torch.optim.lr_scheduler.StepLR(optimizer_G, step_size=30, gamma=0.1)
scheduler_D = torch.optim.lr_scheduler.StepLR(optimizer_D, step_size=30, gamma=0.1)

In [32]:
# 重新加载数据集
data_loader = get_data_loader(data_root="E:/Code/GAN/data/Plaster_side", batch_size=64)

In [33]:
# 重新训练模型
num_epochs = 10000
for epoch in range(num_epochs):
    for batch_idx, real_images in enumerate(dataloader):
        real_images = real_images.to(device)
        batch_size = real_images.size(0)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # 训练判别器
        optimizer_D.zero_grad()
        real_outputs = discriminator(real_images)

        # 创建与 real_outputs 相同维度的 real_labels
        real_labels = torch.ones_like(real_outputs).to(device)

        real_loss = criterion(real_outputs, real_labels)
        real_loss.backward()

        noise = torch.randn(batch_size, 100, 1, 1).to(device)
        fake_images = generator(noise)
        fake_outputs = discriminator(fake_images.detach())

        # 创建与 fake_outputs 相同维度的 fake_labels
        fake_labels = torch.zeros_like(fake_outputs).to(device)

        fake_loss = criterion(fake_outputs, fake_labels)
        fake_loss.backward()
        optimizer_D.step()

        # 训练生成器
        optimizer_G.zero_grad()
        fake_outputs = discriminator(fake_images)

        # 创建与 fake_outputs 相同维度的 real_labels（因为生成器希望生成的图像被判别为真实的）
        real_labels = torch.ones_like(fake_outputs).to(device)

        g_loss = criterion(fake_outputs, real_labels)
        g_loss.backward()
        optimizer_G.step()
    # 学习率调度器更新学习率
    scheduler_G.step()
    scheduler_D.step()

    # 记录损失到TensorBoard
    global_step = epoch * len(dataloader) + batch_idx
    writer.add_scalar('Loss/Real Loss', real_loss.item(), global_step)
    writer.add_scalar('Loss/Fake Loss', fake_loss.item(), global_step)
    writer.add_scalar('Loss/Generator Loss', g_loss.item(), global_step)

In [34]:
# 保存最佳模型权重
torch.save(generator.state_dict(), "best_generator.pth")

In [35]:
# 指定保存图像的目录
output_dir = 'generated_images'

# 如果目录不存在，创建它
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
# 使用最佳模型生成图像

generator.load_state_dict(torch.load("best_generator.pth"))
generator.eval()  # 设置为评估模式
with torch.no_grad():
    num_samples = 10
    for i in range(num_samples):
        noise = torch.randn(1, 100, 1, 1).to(device)
        fake_image = generator(noise)
        # 将生成的图片保存到指定目录
        save_image(fake_image, os.path.join(output_dir, f'generated_image_{i}.png'))

In [36]:
# 关闭SummaryWriter
writer.close()

In [1]:
%load_ext tensorboard
%tensorboard --logdir 'runs'

Reusing TensorBoard on port 6006 (pid 30316), started 3 days, 11:52:25 ago. (Use '!kill 30316' to kill it.)